# 1. Loading Tags to RDS

In [1]:
from sqlalchemy import create_engine, text
import sqlalchemy as sqla
import pandas as pd
import boto3
import warnings
import numpy as np
import importlib
import Loaders
importlib.reload(Loaders)
from Loaders import SQLLoader
warnings.filterwarnings("ignore")

# If requiered
import sqlalchemy as sa
from sqlalchemy import Float, Double
from sqlalchemy.dialects import oracle

In [2]:
ssm = boto3.client('ssm')
dbhost = ssm.get_parameter(Name='/dbtester/dbhost', WithDecryption=True)['Parameter']['Value']
dbport = ssm.get_parameter(Name='/dbtester/dbport', WithDecryption=True)['Parameter']['Value']
dbuser = ssm.get_parameter(Name='/dbtester/dbuser', WithDecryption=True)['Parameter']['Value']
dbpass = ssm.get_parameter(Name='/dbtester/dbpass', WithDecryption=True)['Parameter']['Value']
dbname = ssm.get_parameter(Name='/dbtester/dbname', WithDecryption=True)['Parameter']['Value']

# A. Creating Loaders

In [ ]:
file_tags= 'app-data/tags.csv'
dbengine = 'oracle+oracledb'
dbtable  = 'tags'
dtype    = {"userId": str, "movieId": str, "tag": str, "timestamp": str}
dtype_db = {
    "idx":sa.FLOAT,
    "userId":sa.FLOAT,
    "movieId":sa.FLOAT,
    "tag":sqla.types.VARCHAR(1000),
    "timestamp":sa.FLOAT,
    "_insert":sa.FLOAT,
    "_insert_time":sa.FLOAT,
    "_update":sa.FLOAT,
    "_update_time":sa.FLOAT,
    "_delete":sa.FLOAT,
    "_delete_time":sa.FLOAT,
}

tloader = SQLLoader(
    dbengine,
    dbhost,
    dbuser,
    dbpass,
    dbport,
    dbname,
    dbtable,
    dtype_db = dtype_db)
tloader.load_file(file_tags, dtype=dtype, date_fields=[])
display(tloader.df.head(3), tloader.status())

,userId,movieId,tag,timestamp,_insert,_insert_time,_update,_update_time,_delete,_delete_time
0,3,260,classic,1439472355,0,0,0,0,0,0
1,3,260,sci-fi,1439472256,0,0,0,0,0,0
2,4,1732,dark comedy,1573943598,0,0,0,0,0,0


{'state': 'AVAILABLE',
 'iteration': 0,
 'inserted': 0,
 'updated': 0,
 'deleted': 0,
 'indb': 79470}

tloader.drop_table()

--------

# B. SQL Loading
### B1. Full-Load

In [ ]:
# THE UID LOAD MUST CONTAIN ONLY INSERTS, THIS WAY THE TABLE CAN BE CREATED.
tloader.iud(inserts=100, updates=0 , deletes=0, max_registers=2000)

▶[RUNNING] DB:81470 | I:2000, U:0, D:0 | I/s:1930, U/s:0, D/s:0 | Loop:95.00%, DF:0.18%: 100%|██████████| 20/20 [00:01<00:00, 19.26it/s]

[AVAILABLE] DB:81470 | I:2000, U:0, D:0 | I/s:1930, U/s:0, D/s:0 | Loop:95.00%, DF:0.18%


In [5]:
tloader.status()

{'state': 'AVAILABLE',
 'iteration': 20,
 'inserted': 2000,
 'updated': 0,
 'deleted': 0,
 'indb': 81470}

### B2. CDC (Inserts, Updates, Deletes)

In [ ]:
tloader.iudx(inserts=100, updates=10, deletes=1)

·[RUNNING] DB:98588 | I:20300, U:1830, D:183 | I/s:488, U/s:49, D/s:5 | Loop:1.67%, DF:1.86%

#### Extra

Use 👇 to delete the table
```python
tloader.drop_table()
````